## **Final Project**

### **A. Introduction**

**Team Members**  :
- Livia Amanda Annafiah
- Alfarabi
- Badriah Nursakinah

**Dataset**       : [Airline Reviews](https://www.kaggle.com/datasets/juhibhojani/airline-reviews/data)  

**Hugging Face**  : [Link](https://huggingface.co/spaces/liviamanda/FlightBuddy)

---

**Problem Statement**  

Choosing the right airline can greatly affect a traveler's overall experience, including comfort, service quality, and in-flight amenities. With many online reviews available, airline passengers often **rely on these reviews** to make informed decisions about which airline to choose. However, the large number of reviews can make it difficult and **time-consuming** to read through and understand the general opinion about an airline.

**FlightBuddy** aims to solve this problem by using advanced Natural Language Processing (NLP) techniques to analyze airline reviews quickly and accurately. By processing and understanding a large number of reviews, FlightBuddy can determine whether the opinions in the reviews are positive or negative.

---

**Objective**  

The main goal of **FlightBuddy** is to improve the decision-making process for travelers by providing personalized airline recommendations based on the analysis of review sentiments. Specifically, FlightBuddy aims to:

- Analyze the sentiment of airline reviews to classify them as positive or negative, with accuracy serving as the metric.
- Recommend five airlines with similar positive characteristics for users who have seen favorable reviews.
- Suggest top-rated alternative airlines for users who have encountered negative experiences, ensuring they have better options for future travel.

***This notebook focuses on developing the recommender system using cosine similarity.***

### **B. Libraries**

The following libraries are used for this analysis:

In [1]:
# Import libraries for data loading and manipulation
import pandas as pd
import numpy as np

# Import libraries for recommendation system
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import skew


# Import library to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### **C. Data Loading**

To build this recommender system, only the `airline_name` and all of the rating columns are utilized. **PostgreSQL** is employed to query and select these necessary columns. The ratings are then averaged and grouped by airline name to create the recommendations.

```sql
-- Create new table
CREATE TABLE airline_reviews (
    id SERIAL PRIMARY KEY,
    airline_name VARCHAR(255),
    overall_rating VARCHAR,
    review_title VARCHAR(255),
    review_date VARCHAR(50),
    verified BOOLEAN,
    review TEXT,
    aircraft VARCHAR(50),
    type_of_traveller VARCHAR(50),
    seat_type VARCHAR(50),
    route VARCHAR(255),
    date_flown VARCHAR(50),
    seat_comfort FLOAT,
    cabin_staff_service FLOAT,
    food_beverages FLOAT,
    ground_service FLOAT,
    inflight_entertainment FLOAT,
    wifi_connectivity FLOAT,
    value_for_money FLOAT,
    recommended VARCHAR(10));

-- Insert data to the table
COPY airline_reviews
FROM '/tmp/airline_review_cleaned.csv'
DELIMITER ','
CSV HEADER;

-- Check the table
SELECT * FROM airline_reviews

-- Select only airline and columns of rating
COPY (
    SELECT 
        airline_name,
        AVG(seat_comfort) AS avg_seat_comfort,
        AVG(cabin_staff_service) AS avg_cabin_staff_service,
        AVG(food_beverages) AS avg_food_beverages,
        AVG(ground_service) AS avg_ground_service,
        AVG(inflight_entertainment) AS avg_inflight_entertainment,
        AVG(wifi_connectivity) AS avg_wifi_connectivity,
        AVG(value_for_money) AS avg_value_for_money
    FROM 
        airline_reviews
    GROUP BY 
        airline_name
) TO '/tmp/avg_ratings_per_airline.csv' WITH CSV HEADER;

```

After successfully querying the data, the resulting CSV file is read and displayed for further analysis.

In [2]:
# Read CSV file
df = pd.read_csv('avg_ratings_per_airline.csv')

# Display the first 5 rows
df.head()

,airline_name,avg_seat_comfort,avg_cabin_staff_service,avg_food_beverages,avg_ground_service,avg_inflight_entertainment,avg_wifi_connectivity,avg_value_for_money
0,Air Costa,5.000000,4.000000,3.000000,4.00,2.00,1.00,5.00
1,UP by El Al,2.000000,2.250000,2.000000,2.50,2.00,1.00,2.50
2,Iraqi Airways,3.000000,2.750000,2.750000,2.50,2.75,1.00,3.00
3,Asiana Airlines,3.643286,4.029729,3.819823,3.65,2.96,1.57,3.83
4,Air Tahiti Nui,2.461307,2.996890,2.605181,1.63,2.25,1.11,2.40


In [3]:
# Display the last 5 rows
df.tail()

,airline_name,avg_seat_comfort,avg_cabin_staff_service,avg_food_beverages,avg_ground_service,avg_inflight_entertainment,avg_wifi_connectivity,avg_value_for_money
492,Copa Airlines,2.160389,2.383309,1.997148,1.570000,1.850000,1.08,1.65000
493,Eastern Airways,2.496214,3.021872,2.515598,1.555556,1.793651,1.00,2.31532
494,Aegean Airlines,2.793286,3.201013,2.766253,2.510000,2.170000,1.55,2.49000
495,WOW air,2.019470,2.448445,1.995721,1.870000,1.570000,1.05,2.00000
496,Jazz,3.200000,4.600000,3.600000,4.000000,2.200000,1.60,3.80000


The dataframe consists of **airline names** and the **average ratings** across various service categories as provided by reviewers. These categories include seat comfort, cabin staff service, food and beverages, ground service, inflight entertainment, wifi connectivity, and overall value for money.

Next, the method `info` is used to check the summary of the dataframe.

In [4]:
# Check data summary
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   airline_name                497 non-null    object 
 1   avg_seat_comfort            497 non-null    float64
 2   avg_cabin_staff_service     497 non-null    float64
 3   avg_food_beverages          497 non-null    float64
 4   avg_ground_service          497 non-null    float64
 5   avg_inflight_entertainment  497 non-null    float64
 6   avg_wifi_connectivity       497 non-null    float64
 7   avg_value_for_money         497 non-null    float64
dtypes: float64(7), object(1)
memory usage: 31.2+ KB


The dataframe contains information on **497** airlines, each with average ratings across seven service categories. All columns are complete with no missing values, and the ratings are stored as floating-point numbers. The `airline_name` column is the only one with an object data type, while the others are numerical. The dataframe efficiently captures and organizes the reviewers' feedback on various aspects of airline service.

Then, the summary of descriptive statistics is used to understand the central tendency, dispersion, and overall distribution of the average ratings across different service categories for all airlines. This provides insights into the general performance and variability in service quality among the airlines.

In [5]:
# Check descriptive statistictis
df.describe()

,avg_seat_comfort,avg_cabin_staff_service,avg_food_beverages,avg_ground_service,avg_inflight_entertainment,avg_wifi_connectivity,avg_value_for_money
count,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000
mean,2.709990,2.996257,2.614766,2.156304,1.999014,1.113934,2.633069
std,0.708507,0.772564,0.666478,0.953252,0.442455,0.220189,0.928149
min,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000
25%,2.285335,2.487066,2.237688,1.450000,1.777778,1.000000,1.980000
50%,2.618374,2.890199,2.553586,1.960000,2.000000,1.000000,2.508560
75%,3.060389,3.428571,2.851195,2.600000,2.066667,1.120000,3.153846
max,5.000000,5.000000,5.000000,5.000000,4.010000,2.390000,5.000000


Here are the key insights from the descriptive statistics:

- **Average Ratings**: Most service categories have average ratings around 2 to 3, indicating moderate satisfaction.
- **Highest and Lowest**: Cabin staff service has the highest mean rating (2.99), while wifi connectivity has the lowest (1.11).
- **Variability**: Ground service ratings show the most variability (std 0.95), while wifi connectivity is the least variable (std 0.22).
- **Range**: Ratings range from 1 to 5 in most categories, with some reaching a low of 0 in food and beverages and inflight entertainment.
- **Median and Quartiles**: Half of the airlines have ratings between 2.29 and 3.06 for seat comfort, with similar ranges for other categories.

After that, to ensure there are no missing values as previously mentioned, the `isnull().sum()` function is used to verify.

In [6]:
# Check missing values
df.isnull().sum()

airline_name                  0
avg_seat_comfort              0
avg_cabin_staff_service       0
avg_food_beverages            0
avg_ground_service            0
avg_inflight_entertainment    0
avg_wifi_connectivity         0
avg_value_for_money           0
dtype: int64

There are indeed no missing values in the dataset.

Then, the `duplicated().sum()` function is used to ensure there are no duplicated rows in the dataset.

In [7]:
# Check total of duplicated rows
df.duplicated().sum()

0

The dataset does not contain any duplicated rows, indicating that it's clean and ready for further analysis.

### **D. Recommender System**

To achieve the goals of this recommender system, the approach is to recommend **five similar airlines** with the same characteristics when reviews are **positive**. In contrast, if the review is **negative**, the app will recommend the **top five airlines** rated by all reviewers.

The `cosine_similarity` metric is chosen for determining similarity between airlines because it measures the cosine of the angle between two vectors, providing a measure of similarity between them. In this context, the vectors represent the characteristics or features of airlines. By using `cosine_similarity`, the recommender system can identify airlines with similar characteristics based on the positive reviews and recommend them accordingly. Additionally, it can also rank airlines based on their overall ratings when the review is negative, ensuring that users receive top-rated options.

The initial step involves standardizing the rating columns. Skewness is assessed to determine whether the columns are normally distributed. If the columns exhibit **normal distribution**, `StandardScaler` will be applied. Conversely, if the columns display **skewness**, `MinMaxScaler` will be used.

In [8]:
# Define the rating columns
rating_columns = ['avg_seat_comfort', 'avg_cabin_staff_service', 'avg_food_beverages', 'avg_ground_service',
                  'avg_inflight_entertainment', 'avg_wifi_connectivity', 'avg_value_for_money']

# Check skewness
skewness = df[rating_columns].apply(lambda x: skew(x)) 
skewness = skewness[abs(skewness) > 0.5]

# Initialize scaler variable
scaler = None

# Check if there are columns with significant skewness
if skewness.empty:
    scaler = StandardScaler()  # Use StandardScaler if no significant skewness
else:
    scaler = MinMaxScaler()  # Use MinMaxScaler if significant skewness is present

# Normalize the data using the selected scaler
normalized_data = scaler.fit_transform(df[rating_columns])

Once all columns are standardized using either `StandardScaler` or `MinMaxScaler` based on skewness, the next step involves defining the cosine similarity matrix and converting it into a DataFrame for further analysis and processing.

In [39]:
# Compute cosine similarity
similarity_matrix = cosine_similarity(normalized_data)

# Convert the similarity matrix to a DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=df['airline_name'], columns=df['airline_name'])

# Display the DataFrame
similarity_df

airline_name,Air Costa,UP by El Al,Iraqi Airways,Asiana Airlines,Air Tahiti Nui,Wingo,Nordic Regional Airlines,Wizz Air,Shenzhen Airlines,T'Way Air,...,MEGA Maldives Airlines,Bangkok Airways,Royal Brunei Airlines,Monarch Airlines,euroAtlantic Airways,Copa Airlines,Eastern Airways,Aegean Airlines,WOW air,Jazz
airline_name,,,,,,,,,,,,,,,,,,,,,
Air Costa,1.000000,0.914532,0.932000,0.936288,0.879379,0.781143,0.957064,0.854260,0.923962,0.939465,...,0.630527,0.972335,0.953398,0.968419,0.701345,0.842755,0.896340,0.883320,0.903438,0.928036
UP by El Al,0.914532,1.000000,0.986148,0.956692,0.947796,0.916519,0.957896,0.965833,0.940670,0.970631,...,0.786027,0.956377,0.975215,0.939037,0.882564,0.938632,0.934438,0.923162,0.974139,0.928309
Iraqi Airways,0.932000,0.986148,1.000000,0.959607,0.973393,0.940647,0.939840,0.974896,0.973469,0.977434,...,0.763894,0.959802,0.983827,0.954274,0.875166,0.964752,0.965309,0.933703,0.983080,0.918643
Asiana Airlines,0.936288,0.956692,0.959607,1.000000,0.951365,0.882967,0.944829,0.942255,0.944038,0.967679,...,0.768267,0.972197,0.993041,0.960452,0.885740,0.934445,0.937727,0.988678,0.967464,0.987815
Air Tahiti Nui,0.879379,0.947796,0.973393,0.951365,1.000000,0.967348,0.873112,0.984563,0.987621,0.975269,...,0.837373,0.937343,0.966710,0.949839,0.915130,0.989361,0.992681,0.951700,0.990433,0.917246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copa Airlines,0.842755,0.938632,0.964752,0.934445,0.989361,0.986849,0.863021,0.990464,0.962284,0.958526,...,0.851242,0.914711,0.950644,0.915371,0.933647,1.000000,0.977099,0.944956,0.983857,0.892073
Eastern Airways,0.896340,0.934438,0.965309,0.937727,0.992681,0.955731,0.873880,0.972776,0.994748,0.981390,...,0.834422,0.949578,0.957871,0.967069,0.879740,0.977099,1.000000,0.930631,0.985797,0.911014
Aegean Airlines,0.883320,0.923162,0.933703,0.988678,0.951700,0.895179,0.896509,0.940397,0.927133,0.944645,...,0.791047,0.938079,0.973172,0.928168,0.916176,0.944956,0.930631,1.000000,0.957570,0.973576


The DataFrame `similarity_df` contains rows and columns corresponding to each airline's name, with the values representing the cosine similarity scores between every pair of airlines. This DataFrame offers a comprehensive view of the similarity relationships among different airlines, reflecting their shared characteristics or features.

The next step involves creating the recommender system, consist of two different functions:

1. **Recommendation based on Positive Reviews:**
   - If a review is positive, the system will recommend five airlines with similar characteristics to the reviewed airline.

2. **Recommendation based on Negative Reviews:**
   - If a review is negative, the system will recommend the top five airlines based on overall reviewer ratings.

These functions will enhance the user experience by providing tailored recommendations depending on the sentiment of the reviews.

In [35]:
# Create function to recommend 5 similar airlines for positive reviews
def recommendation_positive(airline, n_recommendations=5):
    # Get the similarity scores for the specified airline with all others
    similar_scores = similarity_df[airline].sort_values(ascending=False)

    # Remove the airline itself from the recommendation
    similar_scores = similar_scores.drop(airline)

    # Reset index to ensure alignment with df
    similar_scores.reset_index(drop=True, inplace=True)

    # Get the top N similar airlines based on positive reviews
    top_positive_airlines = similar_scores.head(n_recommendations)

    # Get the airline names corresponding to the top positive airlines
    top_airlines_with_names = df.loc[top_positive_airlines.index, 'airline_name']

    # Create a DataFrame with airline names and similarity scores
    top_positive_df = pd.DataFrame({'Airline': top_airlines_with_names.values, 'Similarity Score': top_positive_airlines.values})

    return top_positive_df

# Create function to recommend top 5 airlines for negative reviews
def recommendation_negative(airline, n_recommendations=5):
    # Calculate the mean ratings for all airlines
    mean_ratings = df[rating_columns].mean(axis=1)

    # Get the top N airlines based on overall positive reviews
    top_airlines = mean_ratings.nlargest(n_recommendations)

    # Get the airline names corresponding to the top airlines
    top_airlines_with_names = df.loc[top_airlines.index, 'airline_name']

    # Create a DataFrame with airline names and mean ratings
    top_airlines_df = pd.DataFrame({'Airline': top_airlines_with_names.values, 'Mean Rating': top_airlines.values})

    return top_airlines_df

The functions for recommending airlines based on positive and negative reviews are now defined.

### **E. Test the Recommender System**

To verify the functionality of the system, a random airline name will be input into the function. For this analysis, let's use `TUS Airways` as an example.

In [36]:
# Test the function by getting similar airlines to 'TUS Airways'
similar_airlines_to_tus = recommendation_positive('TUS Airways')

# Show result
similar_airlines_to_tus

,Airline,Similarity Score
0,Air Costa,0.997161
1,UP by El Al,0.996809
2,Iraqi Airways,0.996179
3,Asiana Airlines,0.996179
4,Air Tahiti Nui,0.996179


When the `recommendation_positive` function is applied, it will return five airlines with similar characteristics to `TUS Airways` along with their respective similarity scores.

In [37]:
# Test the function by getting similar airlines to 'TUS Airways'
top_5_airline = recommendation_negative('TUS Airways')

# Show result
top_5_airline

,Airline,Mean Rating
0,China Southern Airlines,4.052282
1,Grand Cru Airlines,4.000000
2,Sun-Air,4.000000
3,Lubeck Air,4.000000
4,JetsuiteX,4.000000


When the `recommendation_negative` function is applied, it will consistently offer the top five airlines with high ratings across all reviewers, along with their corresponding mean rating scores, regardless of the specific airline names inputted into the function.

### **F. Conclusion**

In conclusion, the recommender system successfully generates recommendations based on review sentiment. It accurately suggests five similar airlines for positive reviews, considering characteristics and similarity scores, and identifies the top five airlines with high ratings across all reviewers for negative reviews, ensuring consistent and reliable suggestions irrespective of the inputted airline names.